# 1. Demonstrate Ensemble of Vanilla CNN


In [4]:
# IF USING GOOGLE COLAB
# from google.colab import drive
# drive.mount("/content/gdrive")

# Import libraries and ignore warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
np.random.seed(2)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import tensorflow as tf
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image

import os

Using TensorFlow backend.


# Import data as pixels

In [5]:
# Read the annotations file that contains the label and the image file name
labels = pd.read_csv('../data/raw/annotations.csv', header=None, names=['fname','label'])

# Shuffle data
labels = labels.sample(frac=1).reset_index()

# Use a list comprehension to loop over image file names and import one by one and store pixel values
x = np.array([image.img_to_array(image.load_img('../data/raw/all/'+fname.lower(), target_size=(128, 128))) for fname in labels['fname']])

# Because the names are strings, the neural network only takes in numerical formats so we will one-hot encode the label
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels['label'])
y = integer_encoded

# Define an architecture

In [6]:
def build_arch_01():
    
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Feed to a densily connected layer for prediction
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=['acc'])
    
    return model

In [7]:
def build_arch_02():
    
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))

    # Feed to a densily connected layer for prediction
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=['acc'])
    
    return model

In [8]:
def build_arch_03():
    
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.05))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.05))

    # Feed to a densily connected layer for prediction
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.15))

    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.15))

    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=['acc'])
    
    return model

# Run experiment for 10 Stratified K-Fold

In [14]:
# All classification reports will be added here. When we are done we can average the f1 scores
reports = []

# Apply stratified K-fold ith 10 splits. Stratified means the same distribution of classes than the whole dataset
kf = StratifiedKFold(n_splits=10)

# Just for printing purposes
id = 1

for train_index, test_index in kf.split(x,y):
    print('Kfold iteration {}/10'.format(id))
    print('Total images: {} ---- Train images: {} ---- Test images: {}'.format(len(x),len(train_index),len(test_index)))

    id += 1 
    
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
        
    datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=10, # randomly rotate images in the range (degrees, 0 to 180)
                                 width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
                                 height_shift_range=0.1, 
                                 shear_range=0.1,
                                 zoom_range=0.1)
    
    datagen.fit(X_train)

    # Adjust the learning rate over time. (Like we saw in class!)    
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
    
    # Ensemble Learning
    ensemble_predictions = {}
    for i in range(2):
        print("training model:",i)
        model = build_arch_01()
        history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 16), epochs = 30, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 16,
                              callbacks=[learning_rate_reduction])
        
        y_pred_test = model.predict(X_test)
        y_pred_test = y_pred_test.astype(int) # Convert float to int
        y_pred_test = y_pred_test.reshape(y_pred_test.shape[0]) # Reshape

        ensemble_predictions['build_arch_01_'+str(i)] = y_pred_test

    for i in range(2):
        print("training model:",i)
        model = build_arch_02()
        history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 16), epochs = 20, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 16,
                              callbacks=[learning_rate_reduction])
        
        y_pred_test = model.predict(X_test)
        y_pred_test = y_pred_test.astype(int) # Convert float to int
        y_pred_test = y_pred_test.reshape(y_pred_test.shape[0]) # Reshape

        ensemble_predictions['build_arch_02_'+str(i)] = y_pred_test

    for i in range(3):
        print("training model:",i)
        model = build_arch_03()
        history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 16), epochs = 20, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 16,
                              callbacks=[learning_rate_reduction])
        
        y_pred_test = model.predict(X_test)
        y_pred_test = y_pred_test.astype(int) # Convert float to int
        y_pred_test = y_pred_test.reshape(y_pred_test.shape[0]) # Reshape

        ensemble_predictions['build_arch_03_'+str(i)] = y_pred_test

    ensemble_df = pd.DataFrame(ensemble_predictions)

    temp_test = ensemble_df.mode(axis=1)[0]

    reports.append(classification_report(y_test, temp_test, output_dict=True))

In [ ]:
# We loop over all reports (1 per fold) and then compute the average of all weighted f1 scores
final_f1_score = np.mean([rep['weighted avg']['f1-score'] for rep in reports])

print('Final F1-Score is: {}%'.format(np.round(final_f1_score*100,2)))